# Day 22
https://adventofcode.com/2018/day/22

In [1]:
import aocd
data = aocd.get_data(year=2018, day=22)

In [2]:
from collections import deque
from dataclasses import dataclass
from heapq import heapify, heappush, heappop
from typing import Dict, Set
import regex as re

#### Part 1: Identify the risk level of the smallest rectangle containing the cave mouth and the target.

In [3]:
@dataclass(frozen=True, order=True)
class Point():
    y: int
    x: int
        
    @classmethod
    def from_regex_groups(cls, groups):
        x, y = [int(val) for val in groups]
        return cls(y, x)
    
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)
    
    @property
    def neighbours(self):
        neighbours = [self + direction for direction in (Point(-1, 0), Point(0, 1), Point(1, 0), Point(0, -1))]
        return [neighbour for neighbour in neighbours if neighbour.x >=0 and neighbour.y >= 0]

In [4]:
depth = int(re.compile('depth: (\d+)').findall(data)[0])
target = Point.from_regex_groups(re.compile('target: (\d+),(\d+)').findall(data)[0])

In [5]:
@dataclass(frozen=True)
class Region():
    position: Point
    geologic_index: int
        
    @classmethod
    def extend_cave_system(cls, location, cave_system):
        geologic_index = 0
        
        if location != Point(0, 0) and location != target:
            if location.y == 0:
                geologic_index = location.x * 16807
            elif location.x == 0:
                geologic_index = location.y * 48271
            else:
                west = cave_system.get(Point(location.y, location.x-1))
                north = cave_system.get(Point(location.y-1, location.x))
                geologic_index = west.erosion_level * north.erosion_level
        
        region = Region(location, geologic_index)
        cave_system[location] = region
        return region
    
    @property
    def erosion_level(self):
        return (self.geologic_index + depth) % 20183
    
    @property
    def risk_level(self):
        return self.erosion_level % 3
    
    @property
    def terrain(self):
        return ('rocky', 'wet', 'narrow')[self.risk_level]

In [6]:
@dataclass
class CaveSystem():
    system: Dict[Point, Region]
    
    @property
    def risk_level(self):
        return sum(region.risk_level for region in self.system.values())
    
    def get(self, location):
        if location in self.system:
            return self.system[location]
        if location.x < 0 or location.y < 0:
            return None
        return self.add(location)
    
    def add(self, location):
        geologic_index = 0

        if location != Point(0, 0) and location != target:
            if location.y == 0:
                geologic_index = location.x * 16807
            elif location.x == 0:
                geologic_index = location.y * 48271
            else:
                west = self.get(Point(location.y, location.x-1))
                north = self.get(Point(location.y-1, location.x))
                geologic_index = west.erosion_level * north.erosion_level

        region = Region(location, geologic_index)
        self.system[location] = region
        return region
    
    def extend(self, target):
        for location in (Point(y, x) for x in range(target.x+1) for y in range(target.y+1)):
            self.add(location)

In [7]:
cave = CaveSystem({})
cave.extend(target)
p1 = cave.risk_level
print(f'Part 1: {p1}')

Part 1: 8681


#### Part 2: Determine the quickest route to reach the target

In [8]:
INACCESSIBLE_TERRAIN = {
    'climbing gear': 'narrow',
    'torch': 'wet',
    'neither': 'rocky'
}

@dataclass(frozen=True, order=True)
class Equipment():
    name: str
    
    @property
    def inaccessible_terrain(self):
        return INACCESSIBLE_TERRAIN[self.name]
    
    def is_valid_in_terrain(self, terrain):
        return terrain != self.inaccessible_terrain
    
    def switch(self, terrain):
        return next(Equipment(equip) for equip, inaccess in INACCESSIBLE_TERRAIN.items()
                    if equip != self.name and inaccess != terrain)

In [9]:
@dataclass(frozen=True, order=True)
class SearchState():
    minutes: int
    position: Point
    equipment: Equipment
    
    @classmethod
    def initial_state(cls):
        return cls(0, Point(0, 0), Equipment('climbing gear'))
    
    def accessible_neighbours(self, cave_system):
        return (
            loc for loc in self.position.neighbours
            if self.equipment.is_valid_in_terrain(cave_system.get(loc).terrain)
        )
    
    def possible_next_steps(self, cave_system):
        for location in self.accessible_neighbours(cave_system):
            yield SearchState(self.minutes + 1, location, self.equipment)
        
        equip = self.equipment.switch(cave_system.get(self.position).terrain)
        yield SearchState(self.minutes + 7, self.position, equip)

In [10]:
def find_shortest_route(cave):
    best_routes = dict()
    queue = [SearchState.initial_state()]
    heapify(queue)
    
    while len(queue) > 0:
        state = heappop(queue)
        
        if state.position == target:
            return state.minutes
        
        best_key = (state.position, state.equipment)
        if (best_key not in best_routes) or state.minutes < best_routes[best_key]:
            best_routes[best_key] = state.minutes
            for move in state.possible_next_steps(cave):
                heappush(queue, move)

In [11]:
p2 = find_shortest_route(cave)
print(f'Part 2: {p2}')

Part 2: 1070
